# import packages

In [10]:
import math
import numpy as np

import pandas as pd
import os
import csv

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split

from torch.utils.tensorboard import SummaryWriter

# Some Utilty Functions

In [11]:
def same_seed(seed):
    '''Fixed random number generator from reproducibility'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training and validation sets'''
    valid_set_size = int(len(data_set) * valid_ratio)
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator = torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evalueation mode
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())
    preds = torch.cat(preds, dim = 0).numpy()
    return preds

# Dataset

In [22]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Neural Network Model

In [23]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify the model's structure, be aware of dimension.
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x
    

# Feature Selection

In [24]:
def select_feat(train_data, valid_data, test_data, select_all = True):
    '''Select useful features to perform regression'''
    y_train, y_valid = train_data[:, -1], valid_data[:, -1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:, :-1], valid_data[:, :-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [0, 1, 2, 3, 4] # TODO: Select suitable feature columns

    return raw_x_train[:, feat_idx], raw_x_valid[:, feat_idx], raw_x_test[:, feat_idx], y_train, y_valid


# Training Loop

In [42]:
def trainer(train_loader, valid_loader, model, config, device):
    criterion = nn.MSELoss(reduction='mean') # Define your loss function, don't modify this.
    # Define your optimization algorithm.
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithm.
    # TODO L2 regulaization (optimizer(wight, decay...) or implement by yourself).

    # optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9, weight_decay=0.01)
    optimizer = torch.optim.Adam(
        model.parameters(), 
        lr=config['learning_rate'],  # 基础学习率
        betas=(0.9, 0.999),           # 默认的 beta 参数，用于一阶和二阶矩估计
        weight_decay=0.01             # L2 正则化（权重衰减）
    )

    writer = SummaryWriter() # Writer of tensorboard

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.
    
    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode
        loss_record = []

        # tqdm is a package to visiualize your training progress.
        train_pbar = tqdm(train_loader, position = 0, leave = True)

        for x, y in train_pbar:
            optimizer.zero_grad() # Set gradient to zero.
            x, y = x.to(device), y.to(device) # Move your data to device
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward() # Compute gradient(backpropagation)
            optimizer.step() # Update parameters
            step += 1
            loss_record.append(loss.detach().item())

            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch + 1} / {n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record) / len(loss_record)
        # train_loss_record.append(mean_train_loss)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
        
        mean_valid_loss = sum(loss_record) / len(loss_record)
        # valid_loss_record.append(mean_valid_loss)

        print(f'Epoch [{epoch + 1} / {n_epochs}]: train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saveing model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1

        if early_stop_count > config['early_stop']:
            print("\nModel is not improving, so we ahlt the training seesion.")
            return



# Configurations

In [60]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314, # Your seed number, you can pick your lucky number. :)
    'select_all': True, # Whether to use all features.
    'valid_ratio': 0.2,
    'n_epochs': 3000,
    'batch_size': 256,
    'learning_rate': 1e-5,
    'early_stop': 400,
    'save_path': './models/model.ckpt'
}

# Dataloader

In [61]:
# Set seed for reproducibility
same_seed(config['seed'])

# train_data size: 2699 x 118 (id + 37 states + 16 features x 5 days)
# test_data size: 1078 x 117 (without last day's positive rate)
train_data, test_data = pd.read_csv('./covid.train.csv').values, pd.read_csv('./covid.test.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# print out the data size
print(f"""train_data size: {train_data.shape}
valid_data size: {valid_data.shape}
test_data size: {test_data.shape}
""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), COVID19Dataset(x_valid, y_valid), COVID19Dataset(x_test)

# pytorch data loader loads pytorch dataset into batches
train_loader = DataLoader(train_dataset, batch_size = config['batch_size'], shuffle = True, pin_memory = True)
valid_loader = DataLoader(valid_dataset, batch_size = config['batch_size'], shuffle = True, pin_memory = True)
test_loader = DataLoader(test_dataset, batch_size = config['batch_size'], shuffle = False, pin_memory = True)


train_data size: (2160, 118)
valid_data size: (539, 118)
test_data size: (1078, 117)

number of features: 117


# Start training

In [62]:
model = My_Model(input_dim = x_train.shape[1]).to(device)
trainer(train_loader, valid_loader, model, config, device)

Epoch [1 / 3000]: 100%|██████████| 9/9 [00:00<00:00, 180.00it/s, loss=434]


Epoch [1 / 3000]: train loss: 498.7037, Valid loss: 511.6682
Saveing model with loss 511.668...


Epoch [2 / 3000]: 100%|██████████| 9/9 [00:00<00:00, 193.25it/s, loss=538]


Epoch [2 / 3000]: train loss: 494.8100, Valid loss: 504.1395
Saveing model with loss 504.139...


Epoch [3 / 3000]: 100%|██████████| 9/9 [00:00<00:00, 216.83it/s, loss=521]


...

Epoch [2998 / 3000]: train loss: 1.8525, Valid loss: 3.1526


Epoch [2999 / 3000]: 100%|██████████| 9/9 [00:00<00:00, 198.92it/s, loss=2.02]


Epoch [2999 / 3000]: train loss: 1.8361, Valid loss: 2.1501


Epoch [3000 / 3000]: 100%|██████████| 9/9 [00:00<00:00, 230.84it/s, loss=1.48]

Epoch [3000 / 3000]: train loss: 1.8019, Valid loss: 2.4326


In [63]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w', newline='') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
save_pred(preds, 'pred.csv')

C:\Users\Doog\AppData\Local\Temp\ipykernel_17896\3150603070.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(config['save_path']))
100%|

In [64]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314, # Your seed number, you can pick your lucky number. :)
    'select_all': True, # Whether to use all features.
    'valid_ratio': 0.2,
    'n_epochs': 1,
    'batch_size': 128,
    'learning_rate': 1e-5,
    'early_stop': 400,
    'save_path': './models/model.ckpt'
}

In [65]:
trainer(train_loader, valid_loader, model, config, device)

Epoch [1 / 1]: 100%|██████████| 9/9 [00:00<00:00, 207.64it/s, loss=1.91]

Epoch [1 / 1]: train loss: 1.8660, Valid loss: 2.3464
Saveing model with loss 2.346...
